In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
import nltk
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import *
from transformers import RobertaTokenizer, RobertaForTokenClassification, AlbertTokenizer
from transformers import AutoTokenizer, AutoModel
import transformers
from transformers import get_linear_schedule_with_warmup
from transformers import BertForTokenClassification, AdamW, AlbertForTokenClassification, DistilBertForTokenClassification, DistilBertTokenizer
from seqeval.metrics import f1_score, accuracy_score
from seqeval.metrics import classification_report
import os
import torch.nn.functional as F

/home/lopez/tools/software-mention-extraction/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-02 18:29:52.345960: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 18:29:52.923317: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:
2023-02-02 18:29:52.923412: W tensorflow/compiler/xla/stream_executor/platform/default/dso

In [2]:
data = pd.read_csv("../data/labeled_dfs_working.csv")
data_holdout = pd.read_csv("../data/labeled_dfs_holdout.csv")

In [3]:
data.columns = ['paper_id',"word", "tag", "sentence_id"]
data['word'] = data['word'].astype(str)
# replace 'B-publisher', 'B-url', 'I-publisher','I-url' and reference marker labels by 'O' to keep the tokens
# for a realistic input
data['tag'] = data['tag'].replace(['B-publisher', 'B-url', 'B-bibr', 'B-table', 'B-figure', 'B-formula', 'I-publisher', 'I-url', 'I-bibr', 'I-table', 'I-figure', 'I-formula'], 'O')
data = data[data['tag'].isin(['O', 'B-software', 'B-version', 'I-version','I-software'])]

# note: here originally all the other token/labels pairs are removed, while these other labels should be changed to 'O'
# removing all publisher and url labels simplifies the problem with less ambiguity

data_holdout.columns = ['paper_id',"word", "tag", "sentence_id"]
data_holdout['word'] = data_holdout['word'].astype(str)
# replace 'B-publisher', 'B-url', 'I-publisher','I-url' and reference marker labels by 'O' to keep the tokens
# for a realistic input
data_holdout['tag'] = data_holdout['tag'].replace(['B-publisher', 'B-url', 'B-bibr', 'B-table', 'B-figure', 'B-formula', 'I-publisher', 'I-url', 'I-bibr', 'I-table', 'I-figure', 'I-formula'], 'O')
data_holdout = data_holdout[data_holdout['tag'].isin(['O', 'B-software', 'B-version', 'I-version','I-software'])]


In [4]:
data.shape

(304999, 4)

In [5]:
data_holdout.shape

(2703432, 4)

In [6]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["word"].values.tolist(),
                                                     s["tag"].values.tolist())]
        self.grouped = self.data.groupby("sentence_id").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [7]:
getter = SentenceGetter(data)
getter_holdout = SentenceGetter(data_holdout)

In [8]:
sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
sentences_holdout = [[word[0] for word in sentence] for sentence in getter_holdout.sentences]

In [9]:
labels = [[s[1] for s in sentence] for sentence in getter.sentences]
labels_holdout = [[s[1] for s in sentences_holdout] for sentences_holdout in getter_holdout.sentences]

In [10]:
tag_values = list(set(data["tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [11]:
tag2idx

{'I-software': 0,
 'B-version': 1,
 'O': 2,
 'B-software': 3,
 'I-version': 4,
 'PAD': 5}

In [12]:
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

### Finetune SciBERT

In [13]:
MAX_LEN = 215
bs = 32

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_cased', do_lower_case=False)
model = BertForTokenClassification.from_pretrained('allenai/scibert_scivocab_cased', num_labels=len(tag2idx))
model.to('cuda')

loading file vocab.txt from cache at /home/lopez/.cache/huggingface/hub/models--allenai--scibert_scivocab_cased/snapshots/ddf0be025f8e432a1870e34811997ba6725bf04a/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /home/lopez/.cache/huggingface/hub/models--allenai--scibert_scivocab_cased/snapshots/ddf0be025f8e432a1870e34811997ba6725bf04a/config.json
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_cased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "ab

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31116, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [16]:
def tokenize_label_sentence(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)
        tokenized_sentence.extend(tokenized_word)
        # below originally, it should extend labels under the BIO scheme, so extend tokens 
        # replacing B-label by I-label (otherwise we have B- everywhere at each token and wrong 
        # entity scores)
        #labels.extend([label] * n_subwords)
        # fixing the error 
        labels.extend([label])
        if n_subwords>0:
            for i in range(0, n_subwords-1):
                labels.extend([label.replace("B-", "I-")])
    return tokenized_sentence, labels

In [17]:
tokenized_label_text = [
    tokenize_label_sentence(sent, labs)
    for sent, labs in zip(sentences, labels)
]

tokenized_label_text_holdout = [
    tokenize_label_sentence(sent, labs)
    for sent, labs in zip(sentences_holdout, labels_holdout)
]

In [18]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_label_text]
labels = [token_label_pair[1] for token_label_pair in tokenized_label_text]

tokenized_texts_holdout = [token_label_pair[0] for token_label_pair in tokenized_label_text_holdout]
labels_holdout = [token_label_pair[1] for token_label_pair in tokenized_label_text_holdout]

In [19]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

input_ids_holdout = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_holdout],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [20]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

tags_holdout = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels_holdout],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [21]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

attention_masks_holdout = [[float(i != 0.0) for i in ii] for ii in input_ids_holdout]

In [22]:
# we use the whole working set for training
tr_inputs = input_ids
tr_tags = tags
tr_masks = attention_masks

# the full holdout will be used for evaluation
inputs_holdout = input_ids_holdout
tags_holdout = tags_holdout
masks_holdout = attention_masks_holdout

In [23]:
tr_inputs = torch.tensor(tr_inputs)
tr_tags = torch.tensor(tr_tags)
tr_masks = torch.tensor(tr_masks)

In [24]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

In [25]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

/home/lopez/tools/software-mention-extraction/env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [26]:
epochs = 5
max_grad_norm = 1.0

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

### Training

In [27]:
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    model.train()
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        model.zero_grad()
        outputs = model(b_input_ids, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        loss = outputs[0]
        loss.backward()
        total_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        optimizer.step()
        scheduler.step()
        
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))
    loss_values.append(avg_train_loss)

    model.eval()
    predictions , true_labels = [], []

Epoch:  20%|████████████▍                                                 | 1/5 [04:46<19:07, 286.92s/it]

Average train loss: 0.021132648516151296


Epoch:  40%|████████████████████████▊                                     | 2/5 [09:37<14:27, 289.10s/it]

Average train loss: 0.0023306389849719576


Epoch:  60%|█████████████████████████████████████▏                        | 3/5 [14:29<09:40, 290.46s/it]

Average train loss: 0.0012234478327244746


Epoch:  80%|█████████████████████████████████████████████████▌            | 4/5 [19:20<04:50, 290.45s/it]

Average train loss: 0.0007539731466664724


Epoch: 100%|██████████████████████████████████████████████████████████████| 5/5 [24:08<00:00, 289.77s/it]

Average train loss: 0.0005031775908233138


### Save the model

In [28]:
model_out_address = '../models/scibert_software_sent_working'

if not os.path.exists(model_out_address):
    os.makedirs(model_out_address) 

In [29]:
model_to_save = model.module if hasattr(model, 'module') else model 

In [30]:
output_model_file = os.path.join(model_out_address, "pytorch_model.bin")
output_config_file = os.path.join(model_out_address, "config.json")

In [31]:
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(model_out_address)

('../models/scibert_software_sent_working/vocab.txt',)

### Performace metrics

In [32]:
model.eval();

holdout_inputs = torch.tensor(inputs_holdout)
holdout_tags = torch.tensor(tags_holdout)
holdout_masks = torch.tensor(masks_holdout)

holdout_data = TensorDataset(holdout_inputs, holdout_masks, holdout_tags)
holdout_sampler = SequentialSampler(holdout_data)
holdout_dataloader = DataLoader(holdout_data, sampler=holdout_sampler, batch_size=bs)

In [33]:
# full holdout eval

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print("  Num examples = {}".format(len(holdout_inputs)))
print("  Batch size = {}".format(bs))
for step, batch in enumerate(holdout_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
    with torch.no_grad():
        outputs = model(input_ids,
                        attention_mask=input_mask,)
        logits = outputs[0]
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    label_ids = label_ids.to('cpu').numpy()
    input_mask = input_mask.to('cpu').numpy()
    for i,mask in enumerate(input_mask):
        temp_1 = []
        temp_2 = []
        
        for j, m in enumerate(mask):
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : 
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        y_true.append(temp_1)
        y_pred.append(temp_2)

report = classification_report(y_true, y_pred,digits=4)

***** Running evaluation *****
  Num examples = 114796
  Batch size = 32


In [34]:
print("F1 score: %f"%(f1_score(y_true, y_pred)))
print(report)

F1 score: 0.562856
           precision    recall  f1-score   support

  version     0.4547    0.9310    0.6110       232
 software     0.4121    0.8089    0.5460       649

micro avg     0.4229    0.8411    0.5629       881
macro avg     0.4233    0.8411    0.5631       881

